## Dando acesso a sites para o BOT

Com langchain conseguimos conectar a divervas fontes de Dados, para personalizar mais o BOT, com arquivos de culturas e diretrizes de empresas, perguntas padrão, manuais, video aulas, livros e sites.

No arquivo Atual vou conectar a 3 tipos de fontes de dados:
1. Sites
2. Videos
3. PDFs

In [2]:
import os
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv('GROQ_API_KEY')
os.environ['GROQ_API_KEY'] = api_key

chat = ChatGroq(model='llama-3.3-70b-versatile')

### 1. Conectando a Sites

No caso, site da Asimov Academy, local aonde estou fazendo as aulas

In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://asimov.academy/")
lista_documentos = loader.load()
print(lista_documentos)

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://asimov.academy/', 'title': 'Asimov Academy | A maior escola de Python do Brasil', 'description': 'Cursos de programação em Python e Ciência de Dados com ensino baseado fortemente em projetos. Seja membro e tenha acesso ilimitado!', 'language': 'pt-BR'}, page_content=' Asimov Academy | A maior escola de Python do Brasil     Ir para o conteúdoasimovTrilhasTrilha Aplicações IA com PythonTrilha Dashboards Interativos com PythonTrilha Python OfficeTrilha Visão ComputacionalTrilha Data Science e Machine LearningTrilha Análise e Visualização de DadosTrilha Trading QuantitativoCursosProjetosConteúdos gratuitosPython para iniciantesPython para IAPython para DadosBlogTutoriaisPara empresasEntrar\nTrilhasTrilha Aplicações IA com PythonTrilha Dashboards Interativos com PythonTrilha Python OfficeTrilha Visão ComputacionalTrilha Data Science e Machine LearningTrilha Análise e Visualização de DadosTrilha Trading QuantitativoCursosProjetosConteúdos gratuitosPytho

In [4]:
documento = ''
for doc in lista_documentos:
    documento += doc.page_content
    print(documento)

 Asimov Academy | A maior escola de Python do Brasil     Ir para o conteúdoasimovTrilhasTrilha Aplicações IA com PythonTrilha Dashboards Interativos com PythonTrilha Python OfficeTrilha Visão ComputacionalTrilha Data Science e Machine LearningTrilha Análise e Visualização de DadosTrilha Trading QuantitativoCursosProjetosConteúdos gratuitosPython para iniciantesPython para IAPython para DadosBlogTutoriaisPara empresasEntrar
TrilhasTrilha Aplicações IA com PythonTrilha Dashboards Interativos com PythonTrilha Python OfficeTrilha Visão ComputacionalTrilha Data Science e Machine LearningTrilha Análise e Visualização de DadosTrilha Trading QuantitativoCursosProjetosConteúdos gratuitosPython para iniciantesPython para IAPython para DadosBlogTutoriaisPara empresasEntrar


Inscreva-se




Curso 100% gratuito

Aprenda Python do zero e desenvolva novas habilidades
asimovTrilhasTrilha Aplicações IA com PythonTrilha Dashboards com PythonTrilha Python OfficeTrilha Visão ComputacionalTrilha Data Scie

In [5]:
template = ChatPromptTemplate.from_messages([
    ('system', 'Você é um assistente amigável chamado asmio e tem acesso as seguintes informações para dar suas respostas: {documentos_informados} com girias da Bahia-br'), #Definindo  personalizade
    ('user', '{input}')
])

chain = template | chat
resposta = chain.invoke({
    'documentos_informados': documento, 
    'input': 'Quais as trilhas disponíveis na Asimov?'
})

print(resposta.content)

Existem várias trilhas disponíveis na Asimov, incluindo:

1. Trilha Aplicações IA com Python
2. Trilha Dashboards Interativos com Python
3. Trilha Python Office
4. Trilha Visão Computacional
5. Trilha Data Science e Machine Learning
6. Trilha Análise e Visualização de Dados
7. Trilha Trading Quantitativo

Essas trilhas são organizadas por áreas de conhecimento e habilidades específicas, permitindo que os alunos sejam direcionados a um caminho de aprendizado personalizado e focado em suas necessidades e interesses.


### 2. Conectando ao YouTube

In [6]:
from langchain_community.document_loaders import YoutubeLoader

url = 'https://www.youtube.com/watch?v=XA9rSeen-H4'

loader = YoutubeLoader.from_youtube_url(
    url, 
    language=['pt']
)

lista_documentos = loader.load()

In [7]:
import re
import yt_dlp
from youtube_transcript_api import YouTubeTranscriptApi

def extract_video_id(url):
    """Extrai o ID do vídeo do link do YouTube."""
    match = re.search(r"v=([a-zA-Z0-9_-]+)", url)
    return match.group(1) if match else None

def get_video_description(video_id):
    """Pega a descrição do vídeo usando yt_dlp."""
    ydl_opts = {'quiet': True, 'skip_download': True, 'force_generic_extractor': True}
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(f"https://www.youtube.com/watch?v={video_id}", download=False)
        return info.get('description', '')

def extract_timestamps(description):
    """Extrai os tópicos e timestamps da descrição."""
    pattern = re.compile(r'(\d{1,2}):(\d{2})\s+(.+)')
    timestamps = []
    for match in pattern.findall(description):
        minutes, seconds, title = match
        start_time = int(minutes) * 60 + int(seconds)
        timestamps.append((start_time, f"{minutes}:{seconds}", title.strip()))
    return timestamps

def get_transcript(video_id):
    """Pega a transcrição do vídeo."""
    return YouTubeTranscriptApi.get_transcript(video_id, languages=['pt'])

def group_transcript_by_timestamps(transcript, timestamps):
    """Agrupa a transcrição com base nos tópicos da minutagem."""
    grouped_transcript = {}
    
    for i, (start_time, time_str, title) in enumerate(timestamps):
        end_time = timestamps[i + 1][0] if i + 1 < len(timestamps) else float('inf')
        
        grouped_transcript[(time_str, title)] = [
            entry['text'] for entry in transcript 
            if start_time <= entry['start'] < end_time
        ]
    
    return grouped_transcript

def save_transcript_to_markdown(filename, timestamps, grouped_transcript):
    """Salva a transcrição agrupada em um arquivo .md"""
    with open(filename, 'w', encoding='utf-8') as f:
        # Escreve o sumário no início
        f.write("# Sumário\n\n")
        for _, time_str, title in timestamps:
            f.write(f"- [{time_str} {title}](#{title.lower().replace(' ', '-')})\n")
        
        f.write("\n---\n")  # Separador

        # Escreve a transcrição por tópicos
        for (time_str, title), lines in grouped_transcript.items():
            f.write(f"## {title}\n")
            f.write(f"**[{time_str}]**\n\n")  # Adiciona a minutagem
            f.write("\n".join(lines))
            f.write("\n\n---\n")  # Separador entre seções

# 🔥 EXECUÇÃO
video_url = "https://www.youtube.com/watch?v=i5uMhQLepjE"

video_id = extract_video_id(video_url)
if video_id:
    description = get_video_description(video_id)
    timestamps = extract_timestamps(description)
    transcript = get_transcript(video_id)
    grouped_transcript = group_transcript_by_timestamps(transcript, timestamps)
    
    # Salvar no arquivo video.md
    save_transcript_to_markdown("video.md", timestamps, grouped_transcript)
    print("✅ Transcrição salva em 'video.md'")
else:
    print("❌ Não foi possível extrair o ID do vídeo.")


✅ Transcrição salva em 'video.md'


In [8]:
documento = ''
for doc in lista_documentos:
    documento += doc.page_content
    print(documento)

Mas então cara eu instalei é muito fácil tá se tu quer instalar o de psic aí na tua máquina também e ver por conta própria tu não quer dar os teus dados que vão ser armazenados num servidor da China basicamente o que eu tô usando é o olama tá então olama tu entra aí no site ama.com dá meta tu pode instalar pro macos Linux Windows é o partido comunista da China tu não tem escapatória meu ele aparece um robocop tu instalaria Esse modelo no teu Tesla humanoide imagina tu no Tesla humanoide lá tu no Tesla humanoide Ah vou botar o meu vou botar o meu short meu calção para entrar na piscina vem o Tesla humanoide rasga Tá ligado não tem piscina no comunismo não tem piscina no comunismo não tem tu não vai botar o calção não tem piscina no comunismo não não tem como o cara instalar um de psique no humanoide da Tesla imagina vocês estão maluco scp é aquele roda é compilador é é o compilador do deles ai ai o que o que essa galera fala o que que aconteceu na China em 18 1989 eu não sei cara mas no

In [9]:
template = ChatPromptTemplate.from_messages([
    ('system', 'Vc é assistente show de bola, que possue acesso as seguintes informações para dar suas respostas: {informações}'),
    ('user', '{input}')
])

chain_youtube = template | chat
resposta = chain_youtube.invoke({
    'informações': documento, 
    'input': 'Do que se trata o video'
})

print(resposta.content)

O vídeo parece ser sobre Inteligência Artificial (IA) e modelos de linguagem, especificamente sobre o modelo "Deeps" e sua capacidade de processar e responder a perguntas de forma inteligente.

O criador do vídeo está testando o modelo Deeps com diferentes perguntas e avaliando sua capacidade de responder corretamente. Ele também está comparando o desempenho do modelo Deeps com outros modelos de linguagem, como o modelo da OpenAI.

Além disso, o criador do vídeo também está discutindo sobre a necessidade de hardware poderoso para rodar esses modelos de linguagem, especialmente o modelo Deeps com 671 bilhões de parâmetros, que requer uma grande quantidade de memória RAM e uma GPU de alta velocidade.

Ele também menciona que a Apple está se destacando nesse mercado, pois seus chips, como o M2 Ultra, têm uma memória unificada e uma ultra-fusão que permitem uma velocidade e latência mais rápidas, tornando-os mais adequados para rodar esses modelos de linguagem.

O vídeo parece ser um teste

### 3. Conectando a PDFs

In [14]:
from langchain_community.document_loaders import PyPDFLoader

caminho = 'arquivos\RoteiroViagemEgito.pdf'
loader = PyPDFLoader(caminho)
lista_documentos = loader.load()

<>:3: SyntaxWarning: invalid escape sequence '\R'
<>:3: SyntaxWarning: invalid escape sequence '\R'
C:\Users\lucas\AppData\Local\Temp\ipykernel_22864\70028135.py:3: SyntaxWarning: invalid escape sequence '\R'
  caminho = 'arquivos\RoteiroViagemEgito.pdf'


In [15]:
documento = ''
for doc in lista_documentos:
    documento += doc.page_content
    print(documento)

 
 
 
 
 
 
 
 
 
Resumo da viagem 
 
 
 
 
Um passeio perfeito para qualquer pessoa que deseje 
conhecer a Terra dos Faraós em sua viagem ao Egito. Um 
itinerário completo através dos principais monumentos do 
Vale do Nilo, parando em todos os locais-chave. Nossos 
experientes guias nos oferecerão uma visão inigualável 
do país e de sua cultura antiga, enquanto desfrutamos de 
um nível de luxo em todas as nossas acomodações e 
serviços. Esta fabulosa viagem nos permitirá descobrir o 
Cairo e seus museus, as pirâmides do Antigo Império, os 
túmulos escondidos e templos grandiosos do Novo 
Império e da era Ptolomaica, e cruzar o Nilo em um 
cruzeiro de sonho. 
 
 
 
 
 
 
 
 
 
Resumo da viagem 
 
 
 
 
Um passeio perfeito para qualquer pessoa que deseje 
conhecer a Terra dos Faraós em sua viagem ao Egito. Um 
itinerário completo através dos principais monumentos do 
Vale do Nilo, parando em todos os locais-chave. Nossos 
experientes guias nos oferecerão uma visão inigualável 
do país e

In [16]:
template = ChatPromptTemplate.from_messages([
    ('system', 'Vc é assistente show de bola, que possue acesso as seguintes informações para dar suas respostas: {informações}'),
    ('user', '{input}')
])

chain_youtube = template | chat
resposta = chain_youtube.invoke({
    'informações': documento, 
    'input': 'Por quais cidades passaremos nessa viagem?'
})

print(resposta.content)

Nessa viagem, você passará pelas seguintes cidades:

1. Cairo
2. Assuã
3. Kom Ombo
4. Edfu
5. Luxor
6. Dendera
7. Abydos

Além disso, você também visitará outros locais famosos, como as pirâmides de Gizé, o Museu de Antiguidades Egípcias, o Vale dos Reis, o Templo de Hatshepsut, o Templo de Karnak, entre outros.
